In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("creditcard.csv")
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
df.shape

(284807, 31)

In [4]:
df.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [5]:
df['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [6]:
x=df.drop("Class", axis=1)
y=df['Class']

##### Performing cross validation and Hyperparameter tuning 

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from sklearn.model_selection import KFold, GridSearchCV, train_test_split
import numpy as np

In [11]:
# Data partition
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [16]:
lg_regression=LogisticRegression()
grid_params={"C":10.0**np.arange(-3,3),
            'penalty':['l1','l2']
            }
cv=KFold(n_splits=5,shuffle=False)

In [18]:
classif=GridSearchCV(estimator=lg_regression,param_grid=grid_params,cv=cv,
                     n_jobs=-1,scoring='f1_macro',verbose=2)
classif.fit(x_train,y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   31.5s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  1.1min finished
C:\Users\hp\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                         'penalty': ['l1', 'l2']},
             scoring='f1_macro', verbose=2)

In [21]:
y_prediction=classif.predict(x_test)
print(confusion_matrix(y_test,y_prediction))
print(classification_report(y_test,y_prediction))
print(accuracy_score(y_test,y_prediction))

[[56837    24]
 [   38    63]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56861
           1       0.72      0.62      0.67       101

    accuracy                           1.00     56962
   macro avg       0.86      0.81      0.83     56962
weighted avg       1.00      1.00      1.00     56962

0.9989115550718023


In [26]:
from sklearn.ensemble import RandomForestClassifier
rf_class=RandomForestClassifier()
rf_class.fit(x_train,y_train)
pred_y=rf_class.predict(x_test)
print(confusion_matrix(y_test,pred_y))
print(classification_report(y_test,pred_y))
print(accuracy_score(y_test,pred_y))

[[56854     7]
 [   20    81]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56861
           1       0.92      0.80      0.86       101

    accuracy                           1.00     56962
   macro avg       0.96      0.90      0.93     56962
weighted avg       1.00      1.00      1.00     56962

0.9995259997893332


In [27]:
class_wts=dict({0:1,1:100})

###### Changing the class weight

In [28]:
# Changing the class weights
rf_class=RandomForestClassifier(class_weight=class_wts)
rf_class.fit(x_train,y_train)
pred_y=rf_class.predict(x_test)
print(confusion_matrix(y_test,pred_y))
print(classification_report(y_test,pred_y))
print(accuracy_score(y_test,pred_y))

[[56856     5]
 [   23    78]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56861
           1       0.94      0.77      0.85       101

    accuracy                           1.00     56962
   macro avg       0.97      0.89      0.92     56962
weighted avg       1.00      1.00      1.00     56962

0.9995084442259752


In [37]:
from sklearn.svm import SVC
sv_classif=SVC(class_weight=class_wts)
sv_classif.fit(x_train,y_train)
predictions=sv_classif.predict(x_test)

In [38]:
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test,predictions))

[[56861     0]
 [  101     0]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56861
           1       0.00      0.00      0.00       101

    accuracy                           1.00     56962
   macro avg       0.50      0.50      0.50     56962
weighted avg       1.00      1.00      1.00     56962

0.9982268881008391


C:\Users\hp\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### Under Sampling

In [39]:
from collections import Counter
from imblearn.under_sampling import NearMiss
nm=NearMiss(0.8)
xtrain_nm,ytrain_nm=nm.fit_sample(x_train,y_train)

C:\Users\hp\anaconda3\lib\site-packages\imblearn\utils\_validation.py:635: FutureWarning: Pass sampling_strategy=0.8 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.9 "


In [43]:
print("No.of clases before fit {}".format(Counter(y_train)))
print("No.of clases after fit {}".format(Counter(ytrain_nm)))

No.of clases before fit Counter({0: 227454, 1: 391})
No.of clases before fit Counter({0: 488, 1: 391})


In [45]:
rf_class=RandomForestClassifier()
rf_class.fit(xtrain_nm,ytrain_nm)


RandomForestClassifier()

In [46]:
pred_y=rf_class.predict(x_test)
print(confusion_matrix(y_test,pred_y))
print(classification_report(y_test,pred_y))
print(accuracy_score(y_test,pred_y))

[[39615 17246]
 [    5    96]]
              precision    recall  f1-score   support

           0       1.00      0.70      0.82     56861
           1       0.01      0.95      0.01       101

    accuracy                           0.70     56962
   macro avg       0.50      0.82      0.42     56962
weighted avg       1.00      0.70      0.82     56962

0.6971489765106562


#### Over Sampling

In [47]:
from imblearn.over_sampling import RandomOverSampler


In [48]:
os=RandomOverSampler(0.6)
xtrain_os,ytrain_os=os.fit_sample(x_train,y_train)
print("No.of clases before fit {}".format(Counter(y_train)))
print("No.of clases after fit {}".format(Counter(ytrain_os)))

C:\Users\hp\anaconda3\lib\site-packages\imblearn\utils\_validation.py:635: FutureWarning: Pass sampling_strategy=0.6 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.9 "


No.of clases before fit Counter({0: 227454, 1: 391})
No.of clases before fit Counter({0: 227454, 1: 136472})


In [50]:
rf=RandomForestClassifier()
rf.fit(xtrain_os,ytrain_os)

RandomForestClassifier()

In [51]:
pred_y=rf.predict(x_test)
print(confusion_matrix(y_test,pred_y))
print(classification_report(y_test,pred_y))
print(accuracy_score(y_test,pred_y))

[[56854     7]
 [   19    82]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56861
           1       0.92      0.81      0.86       101

    accuracy                           1.00     56962
   macro avg       0.96      0.91      0.93     56962
weighted avg       1.00      1.00      1.00     56962

0.9995435553526912


#### SMOTETomek

In [53]:
from imblearn.combine import SMOTETomek
sm=SMOTETomek(0.7)
xtrain_sm,ytrain_sm=sm.fit_sample(x_train,y_train)
print("No.of clases before fit {}".format(Counter(y_train)))
print("No.of clases after fit {}".format(Counter(ytrain_sm)))

C:\Users\hp\anaconda3\lib\site-packages\imblearn\utils\_validation.py:635: FutureWarning: Pass sampling_strategy=0.7 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.9 "


No.of clases before fit Counter({0: 227454, 1: 391})
No.of clases after fit Counter({0: 226502, 1: 158265})


In [55]:
rf_sm=RandomForestClassifier()
rf_sm.fit(xtrain_sm,ytrain_sm)

RandomForestClassifier()

In [56]:
pred_y=rf_sm.predict(x_test)
print(confusion_matrix(y_test,pred_y))
print(classification_report(y_test,pred_y))
print(accuracy_score(y_test,pred_y))

[[56847    14]
 [   14    87]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56861
           1       0.86      0.86      0.86       101

    accuracy                           1.00     56962
   macro avg       0.93      0.93      0.93     56962
weighted avg       1.00      1.00      1.00     56962

0.9995084442259752
